# Input RC Pole
## Problem
For some reason there is roloff in my filter at 10 MHz.
- This is probably the pole formed by the input cap together with the switch resistance, which is around 50 * 5e-12 = 25 MHz.
- Given the switch is on half the time this probably reduces the influence to 12.5 MHz

## Solution
Seems to be unsolvable, the only thing you can really do is reduce the source impedance as much as possible to allow quick charging of the caps.

You can though add an extra pole in the amplification circuit so there's amplification happening between 6 and 10 MHz for example if this would be the problematic frequency band.

You thus need a high-speed buffer that can deliver enough current at the right moment, and has a tight feedback loop with the ADC input. So fast voltage drops at the output as a consequence of capacitor current consumption must be immediately filled-up.

# Pre-built differential amplifiers

- They are probably more expensive, but easier to integrate.
- Ideally they have some-kind of tuning possibility to work away the input pole.
- Ideally they have a commonly used package.
- They should take a differential signal and
	- Output a differential signal
	- (still considering) output a single-ended signal -> just in case maybe we'd like to limit ourselves to differential signal transfer, and single-ended measuring once we hit the end. It cheapens our front-end and probably improves signal quality, since I'm not sure whether the system is actually meant for differential signal measurement.

## Examples
### AD: 
#### ADA4940
- differential input to differential output ADC buffer

#### ADA4941
- Differential input to differential output ADC buffer

#### AD8137
[link](https://jlcpcb.com/api/file/downloadByFileSystemAccessId/8604443426939154432)

- differential driver
- common-mode voltage pin
- single-ended to differential and fully differential configurations possible.
- Requires external feedback
- Differential to differential amplification

### TI:
#### OPA356
simple OPAMP, high frequency

#### THS4551
Fully differential amplifier
- Requires external feedback + phase compensation

#### THS452x
Differential amplfiier, example configuration:

![](images/adc_buffer_example_circuit.png)

- GBW: 145 MHz bandwidth: 1 V Amplification
- Small-signal bandwidth: 135 MHz

# Electrical specs
## Pins
- PDn: if pulled-up runs the amplifier.
- Vocm: common mode voltage input

## Pin Electrical Specs

The reference voltage comes from the ADC Vdd, so either
- We make a low-impedance voltage divider, and run a lot of current through it.
- We make a high-impedance divider which we then buffer (probably the better option)
- We use an LDO as a voltage reference

### Input voltages (Vi, Vo, Vocm)
- Between Vs- - 0.7 and Vs+ + 0.7 (so 0-> 4 V max)
- Vid (differential input voltage): 1 V

### Current sourcing / sinking
- Input current: max 10 mA (Vin and Vocm)
- Output current: max 100 mA

## Buffering common mode voltage

# LTSpice

## Differential vs Common mode source resistance

![](images/thevenin_diff_source.png)

This is a way we can create a differential signal with amplitude V_DIFF with an offset of V_CM. However there is no source resistance here and the common-mode impedance is only 50 ohms, where ideally it would be infinity. (impedance to 0 V / Vds should be 0 given there is no "ground" / "common mode" to carry back the signal)

### Thevenin
1. Load Rl: assume infinity -> remove
2. Remove and calculte open circuit voltage: V_CM + V_DIFF
3. Thevenin impedance:
	- Voltage source becomes short
	- Current source becomes open circuit
4. Thevenin's equivalent circuit: a short

### Differential vs Common mode impedance
The ideal scenario: 
- The common-mode impedance should be infinity for AC signals, and 0 for DC signals.
	- No (or extremely small) current flow between the differential and common mode
	- Perfectly balanced signals

Non-ideal scenario:
- There are non-symmetrical effects between your transformer windings.
- These cause assymetric current flows with the common mode.
- These affect the common mode voltage.

Main AC current flowing
- Currents from the AC differential signal 
	- ideally
		- balanced symmetrical signal
		- current flows from one half to the other half of the secondary winding
		- symmetry at the center-tap cancels them out.
	- real-world case
		- imbalances and assymetries in impdeance cause the current to flow through the center tap.


- Single-ended mode
	- Higher resistance between common mode and lines
		- Less distortion in source impedance
		- Less voltage drop due to that
		- More susceptible to leakage or noise
	- Smaller resistance
		- Bias is fixed and won't change with disturbance
		- Zdiff will be different
- The differential-mode impedance should be 0.

## Biasing THS4521 input
The THS4521 input still needs to be biased to a voltage within acceptable limits. 
It can not amplify signals that go below the Vss-, so they need to be biased around a center point.

### Choosing the Rbias value

#### 1. Signal attenuation
- Lower Rbias: lower input impedance, and thus more signal attenuation
- Higher Rbias, higher signal lower signal attenuation

#### 2. DC Offset due to input bias current

Input bias current depends mostly on the THS4521 input bias current.

This input bias current is about 0.80 uA at worst.
- For 1 kOhm: 0.80 mV
- For 100 kOhm: 80 mV offset

#### 3. Thermal noise
- Higher resistance value: more thermal noise

#### 4. High-pass filter (in case of using AC coupling capacitors)
- Your AC coupling capacitance together with your input impedance (R_amplifier_input || R_bias ~= R_bias) to common mode forms a high-pass filter with a zero at 1/(2*pi*R*C)
	- You get a pole (at 1/2*pi*R*C) and a zero (at DC)
	- 1/(2*pi*R*C) = [R=1k, C=100nF] 1.591 kHz which is definitely low enough, making this 10 kOhm lowers it even more.

## Effect of output capacitor
Pole is formed by the closed loop output impdeance $Z_{out, cl}$ and the differential output capacitor $C_{out_diff}$.

A resistor is added to increase the open loop output impedance to 50 ohms for each line.

$$
Z_{\mathrm{out,cl,diff}}(s) \;\approx\; 
\frac{2\,R_{\mathrm{ol}}}{\,1 + A_{\mathrm{ol}}(s)\,\beta(s)\,}
$$

- $A_{ol}$: open loop gain
- $\Beta(s)$: feedback factor (frequency dependent)
- $2*R_{ol}$: open loop resistance added

So a pole in the feedback function appears at $$1/(2*\pi*c_{fb}*r_{fb} = 1/(2*\pi*1e3 * 10e-12 = 15.9 MHz$$


### Example: 10 nF capacitor
- Observed pole at 1.5 MHz
- $C_{F}$ has less impact, feedback factor $\beta$ = 1 and is mainly set by the resistors
So: $$Z_{out,cl,diff} \approx \frac{2*R_{ol}}{1+A_{aol}*\beta}$$
- Because of this the output impdeance is much lower (about 10 ohms)

### Example: 10 pF capacitor

- The feedback factor is influenced by the pole, and starts to decrease.
- As a consequence to this degradation the output impedance starts to increase.
- This will move the pole created by the differential output capacitor and the closed loop impedance to a lower place then initially expected.

## Effect of the output resistor
- Series resistors at the output are meant to limit instability when driving capacitive loads
	- Due to the feedback resistance the voltage drop there will be compensated by the feedback.

## Effect of the feedback capacitor

The feedback capacitor gives a short path for the HF signals to the output.
- Creates a quick path for high-frequency signals (no need to travel through the resistor)
	- Improves phase margin
### Why placing it before the resistor?
- Avoid an RC network with input resistors (might introduce an extra pole in FB loop)



# Reference voltage 

## Isolation resistor at input
### Rule of thumb
- Check datasheet (10-100 ohms)

### Estimation / Calculation
Rough Estimation: Open loop output resistance with the load capacitance forms a pole ($f_{p} = \frac{1}{2*\pi*R_0*C_L}$)

Interaction with $R_{iso}$ adds a zero, so ideally $R_{iso}$ - $C_{L}$-pole must be set below unity-gain bandiwdth.

### Current limiting

Basic check to be done to ensure the output current for the amplifier is not too high:

$$I_{max} = V_{supply} / R_{series}$$

### Tranmission line driving
- Choose the R_out approximately equal to the transmission line impedance if driving a transmission line.

## Resistance at V- to ground
Sometimes a resistor to ground is added here. This creates a steady load for the OPAMP (1-2 mA depending on the output voltage), some opamps need this for stability purposes.

## Effect

# Terminology
## GBP vs Small Signal Gain
### GBP
GBP is simply the product of the amplifiers open loop gain and the frequency at which that gain rolls off.
### Small Signal Gain
The small-signal gain refers to the frequency at which the amplifiers gain drops to -3dB.
